# Imports

In [24]:
import zipfile
import json
from scipy import sparse
import numpy as np
import time
import scipy.sparse.linalg as lg

# Dataset path

In [25]:
datadirectory="C:\\lastfm_test.zip"
dataset = zipfile.ZipFile(datadirectory)

# global variables initialization


In [26]:
songId=0
t=0
k=10
#g=50
#beta=0.2
#categories =["Technical Death Metal"]
#S =0#[]
#id2songJsonMap = []
id2songIdMap = {}
#fhat=[]
#n=5 #top songs to report
#limitingCounter = 1000


# Loading data


In [27]:
start_time = time.time()
for i in dataset.namelist():
    if  i.endswith(".json"): 
        f = json.loads(dataset.read(i).decode("utf-8"))
        if  f["track_id"] not in id2songIdMap.keys():
            id2songIdMap[f["track_id"]] = (songId,i)
            #tempcategories = set(categories)#will keep removing tags from it
            #newtags=[]
            #for tag in f["tags"]:#looping over tags
             #   if int(tag[1]) >g:#filters out those <g
              #      newtags.append(tag)#keeping >g
               #     if tag[0] in tempcategories:#if it is in the tempcategories
                #        tempcategories.remove(tag[0])#removing it
            #f["tags"] = newtags
            #id2songJsonMap.append(f)
            #songTags= [t[0] for t in f["tags"]]
            #if len(list(filter(lambda x: x in categories,songTags )) ) == len(categories):
            #if len(tempcategories)==0:#meaning we encountered all of the tags
             #   S+=1#.append(songId)
              #  fhat.append(1)
            #else:
             #   fhat.append(0)
            songId+=1 
            
print("--- %s seconds ---" % (time.time() - start_time))

--- 17.92873501777649 seconds ---


# initializing the teleportation probablilities


In [28]:
#_fhat=np.array()
#_fhat.shape=(songId,1)
#fhat=(1/len(S))*np.array(fhat)
#fhat=(1/S)*np.array(fhat)

# Building the Adjacency matrix


In [29]:
start_time = time.time()
col=[]
row=[]
data=[]
for i in id2songIdMap.keys():
    f = json.loads(dataset.read(id2songIdMap[i][1]).decode("utf-8"))
    for s in f["similars"]:
        if s[0] in id2songIdMap.keys() and s[1]>=t:
            #similar = json.loads(dataset.read(id2songIdMap[s[0]][1]).decode("utf-8"))
            val=s[1]
            #if i in similar["similars"].keys():
            #    val =max(val,similar["similars"][i])

                
            col.append(id2songIdMap[i][0])
            row.append(id2songIdMap[s[0]][0])
            data.append(val)
            
            #row.append(id2songIdMap[i][0])
            #col.append(id2songIdMap[s[0]][0])
            #data.append(val)
    
    
    

        
print("--- %s seconds ---" % (time.time() - start_time))        


--- 22.846117734909058 seconds ---


In [30]:
A = sparse.csc_matrix((data, (row, col)),shape=(songId, songId))


In [31]:
def maximum(A,B):
    BisBigger = A-B
    BisBigger.data = np.where(BisBigger.data<0,1,0)
    return A-A.multiply(BisBigger)+B.multiply(BisBigger)

In [32]:
W = maximum(A,A.T)


In [103]:
len(list(W.sum(0).flat))
D = sparse.csc_matrix((list(W.sum(0).flat), (range(songId), range(songId))))

In [104]:
L=D-W

# Computing the Smallest K eignvectors

In [ ]:
#w,v = lg.eigs(L,k=k,sigma=0)

In [105]:
#s,v,d = lg.svds(L,k=5,which='SM')
largestEigen= lg.eigs(L,k=1,return_eigenvectors=False)

In [106]:
print (largestEigen)

[ 4002.66891005+0.j]


In [107]:
I = sparse.identity(songId, format='csc')
I.setdiag(largestEigen)

C:\Users\ahmad\Anaconda3\lib\site-packages\numpy\core\numeric.py:482: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


4002.668910046139

In [111]:
B = L-I

# Randomly initializing r


In [112]:
e,v= lg.eigs(B,k=k,return_eigenvectors=True)

In [119]:
v*=-1

In [120]:
print (e*-1)

[ 3997.15114368-0.j -4001.66891005+0.j   -90.81416708+0.j
   -84.30612745+0.j   -82.45367414+0.j   -81.61055030+0.j
   -81.18583872+0.j   -81.03990789+0.j   -81.61055030+0.j
   -80.85525845+0.j]


In [ ]:
for i in range(songId):
    

In [ ]:
r =np.random.rand(A.shape[0])
r=r/np.sum(r)

In [ ]:
#print(len(id2songJsonMap))
print (A.shape)
print(songId)
print (S)
print (songId)

# Power Iterations


In [ ]:
for i in range(100):
    r=A.multiply(beta).dot(r)+fhat*(1-beta)

# Reporting the top n songs


In [ ]:
topIndecs = r.argsort()[-n:][::-1]


In [ ]:
keys= list(id2songIdMap.keys())
for item in topIndecs:
    print (keys[item])

In [ ]:
print (topIndecs)

In [ ]:
A =np.asarray( [[2,3],[1,6]])

In [ ]:
A

In [ ]:
np.maximum(A, A.T)